# Get Azure AD Recent SignIns and display on a Graph

Use an Azure AD Registered Application with **AuditLog.Read.All** permissions to query the most recent 1000 Azure AD SignIns
Using the MSAL.PS PowerShell Module to simplify Microsoft Graph integration using PowerShell

[Reference - https://blog.darrenjrobinson.com/microsoft-graph-using-msal-with-powershell/]( https://blog.darrenjrobinson.com/microsoft-graph-using-msal-with-powershell/ )

To execute this example you will need to register an Azure AD Application with AuditLog.Read.All Application Microsoft Graph permissions then update the cell variables below for;
* your Registered App ClientID
* your Registered App ClientSecret
* your Azure AD TenantID

In [1]:
Install-Module -Name MSAL.PS -Force -AcceptLicense
Import-Module MSAL.PS

$clientID = "<AADRegisteredAppID>"
$clientSecret = (ConvertTo-SecureString "<AADRegisteredAppSecret>" -AsPlainText -Force)
$tenantID = "<AADTenantID>"
$myAccessToken = Get-MsalToken -ClientId $clientID -ClientSecret $clientSecret -TenantId $tenantID -RedirectUri "https://localhost" 

$signIns = Invoke-RestMethod -Method Get `
    -Uri "https://graph.microsoft.com/beta/auditLogs/signIns?`$top=1000" `
    -Headers @{Authorization = "Bearer $($myAccessToken.AccessToken)" } 

$groupedByDate = $signIns.value | Group-Object {$_.createdDateTime.toString("dd-MM-yyyy")} | Select-Object Count, Name
$groupedByDate


Count Name
----- ----
   77 11-07-2020
  171 12-07-2020
  227 13-07-2020
  483 14-07-2020
   92 15-07-2020



## Bar Graph
An example of using New-PlotlyChart for a Bar Chart showing Azure Active Directory SignIns summarised by date. 

In [2]:
$signInsGraphData = [Graph.Bar]::New()
$signInsGraphData.name = "Recent Azure AD SignIns"
$signInsGraphData.x = $groupedByDate.Name

$signInSummaryCount = @()
foreach ($signInCount in $groupedByDate) {
    $signInSummaryCount += $signInCount.Count
}   
$signInsGraphData.y = $signInSummaryCount

$graphLayout = [Layout]::New()
$graphLayout.title = "Recent Azure AD SignIns"
$graphLayout.height = 350

New-PlotlyChart -Layout $graphLayout -Trace @($signInsGraphData) -Title "Recent Azure AD SignIns" | Out-Display


## Scatter Graph
An example of using New-PlotlyChart for a Scatter Chart showing Azure Active Directory SignIns summarised by date. 

In [5]:
$signInsGraphDataLine = [Graph.Scatter]::New()
$signInsGraphDataLine.name = "Recent Azure AD SignIns"
$signInsGraphDataLine.x = $groupedByDate.Name

$signInSummaryCount = @()
foreach ($signInCount in $groupedByDate) {
    $signInSummaryCount += $signInCount.Count
}   
$signInsGraphDataLine.y = $signInSummaryCount

New-PlotlyChart -Layout $graphLayout -Trace @($signInsGraphDataLine) -Title "Recent Azure AD SignIns" | Out-Display